In [1]:
import speech_recognition as sr
import pyttsx3
import datetime
import webbrowser
import random
import pyjokes
import requests
import time
import threading

# Initialize the speech engine (Text to Speech)
engine = pyttsx3.init()

# Set up the OpenWeatherMap API key and base URL
API_KEY = ""
BASE_URL = "http://api.openweathermap.org/data/2.5/weather?"

def speak(text):
    engine.say(text)
    engine.runAndWait()

def listen():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening...")
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)

    try:
        print("Recognizing...")
        command = recognizer.recognize_google(audio).lower()
        print(f"User said: {command}")
        return command
    except sr.UnknownValueError:
        speak("Sorry, I did not understand that.")
        return ""
    except sr.RequestError:
        speak("Sorry, I'm having trouble connecting to the service.")
        return ""

def respond_to_hello():
    responses = [
        "Hello, how can I help you today?",
        "Hi there! What can I do for you?",
        "Hey! How can I assist you?"
    ]
    speak(random.choice(responses))

def tell_time():
    now = datetime.datetime.now()
    current_time = now.strftime("%H:%M")
    speak(f"The current time is {current_time}")

def tell_date():
    now = datetime.datetime.now()
    current_date = now.strftime("%B %d, %Y")
    speak(f"Today's date is {current_date}")

def search_web(query):
    search_url = f"https://www.google.com/search?q={query}"
    webbrowser.open(search_url)
    speak(f"Here are the results for {query}")

def tell_joke():
    joke = pyjokes.get_joke()
    speak(joke)

def fetch_weather(city):
    complete_url = f"{BASE_URL}q={city}&appid={API_KEY}&units=metric"
    response = requests.get(complete_url)
    data = response.json()

    if data["cod"] != "404":
        main_data = data["main"]
        weather_data = data["weather"][0]
        temperature = main_data["temp"]
        weather_description = weather_data["description"]
        speak(f"The current temperature in {city} is {temperature}°C with {weather_description}.")
    else:
        speak("Sorry, I couldn't find the weather for that location.")

def set_alarm(time_str):
    try:
        alarm_time = datetime.datetime.strptime(time_str, "%H:%M")
        now = datetime.datetime.now()
        time_difference = alarm_time - now
        if time_difference.total_seconds() > 0:
            speak(f"Setting an alarm for {time_str}.")
            time.sleep(time_difference.total_seconds())
            speak(f"Alarm! It's {time_str}. Time's up!")
        else:
            speak("The time you entered is in the past. Please set a future time.")
    except ValueError:
        speak("Sorry, I couldn't understand the time format. Please use HH:MM format.")

def run_assistant():
    speak("Hello, I am your voice assistant. How can I help you?")
    
    while True:
        command = listen()

        if "hello" in command:
            respond_to_hello()

        elif "time" in command:
            tell_time()

        elif "date" in command:
            tell_date()

        elif "search" in command:
            query = command.replace("search", "").strip()
            if query:
                search_web(query)
            else:
                speak("Please tell me what to search for.")

        elif "joke" in command:
            tell_joke()

        elif "weather" in command:
            city = command.replace("weather in", "").strip()
            if city:
                fetch_weather(city)
            else:
                speak("Please tell me the city.")

        elif "set alarm" in command:
            time_str = command.replace("set alarm for", "").strip()
            if time_str:
                alarm_thread = threading.Thread(target=set_alarm, args=(time_str,))
                alarm_thread.start()
            else:
                speak("Please tell me the time in HH:MM format.")

        elif "exit" in command or "quit" in command:
            speak("Goodbye! Have a great day!")
            break
        
        else:
            speak("Sorry, I didn't understand that. You can say 'hello', 'time', 'date', 'joke', 'weather', 'set alarm', or ask me to 'search'.")

if __name__ == "__main__":
    run_assistant()


Listening...
Recognizing...
User said: time hello
Listening...
Recognizing...
User said: joke
Listening...
Recognizing...
Listening...
Recognizing...
User said: exit
